## Write Lyrics

we are going to write songs based on previous data.

[Link for Dataset](https://www.kaggle.com/mousehead/songlyrics)

In [1]:
# import necessary libraries
import sys
import numpy as np
import pandas as pd
import sys
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\z003w00f\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\z003w00f\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np

In [2]:
#reading a songs file excel
df = pd.read_csv('Dataset/songdata.csv')
df.head()


,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [3]:
#Combining text columns as remaining are not important
text = df['text'].str.cat(sep='\n').lower()

In [4]:
# Create a sorted list of the characters
chars = sorted(list(set(text)))
print('total chars:', len(chars))

total chars: 50


In [5]:
# Output the length of the corpus
print('length:', len(text))

length: 68056106


In [6]:
# training for first 8000000 text 
text = text[:1000000]
print('length:', len(text))

length: 1000000


In [7]:
# create mapping of all unique chars to integers
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [8]:
n_chars = len(text)
n_vocabs = len(chars)
print("Total characters: ", n_chars)
print("Total vocab: ", n_vocabs)

Total characters:  1000000
Total vocab:  50


In [23]:
seq_length = 40
dataX = []
dataY = []
for i in range(0, n_chars - seq_length , 1):
    seq_in = text[i:i + seq_length]
    seq_out =text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append([char_to_int[seq_out]])

n_patterns = len(dataX)
print("Total patterns: ", n_patterns)

Total patterns:  999960


since LSTMs accept values in the form (no_of_sampels, time_steps, no_of_features), therefore
reshape dataX to this form

In [24]:
# reshape dataX
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X/float(n_vocabs)

# one hot encoding using np_utils
y = np_utils.to_categorical(dataY)

In [25]:
X.shape

(999960, 40, 1)

In [26]:
y.shape

(999960, 50)

In [13]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(LSTM(256))
model.add(Dropout(rate=0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [14]:
# define the checkpoints and callbacks
filepath="SavedModel/weights-imporvement-{epoch: 02d}-{loss: .4f}-from-class.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [16]:
# training the LSTM

model.fit(X, y, epochs=30, batch_size=128, callbacks=callbacks_list) # do try out at diff epoch and batch sizes


Epoch 1/30
999960/999960 [==============================] - 2933s 3ms/step - loss: 2.3228

Epoch 00001: loss improved from inf to 2.32279, saving model to SavedModel/weights-imporvement- 1- 2.3228-from-class.hdf5
Epoch 2/30
999960/999960 [==============================] - 2916s 3ms/step - loss: 1.9330

Epoch 00002: loss improved from 2.32279 to 1.93303, saving model to SavedModel/weights-imporvement- 2- 1.9330-from-class.hdf5
Epoch 3/30
999960/999960 [==============================] - 2892s 3ms/step - loss: 1.7894

Epoch 00003: loss improved from 1.93303 to 1.78942, saving model to SavedModel/weights-imporvement- 3- 1.7894-from-class.hdf5
Epoch 4/30
999960/999960 [==============================] - 2938s 3ms/step - loss: 1.7056

Epoch 00004: loss improved from 1.78942 to 1.70562, saving model to SavedModel/weights-imporvement- 4- 1.7056-from-class.hdf5
Epoch 5/30
999960/999960 [==============================] - 2900s 3ms/step - loss: 1.6453

Epoch 00005: loss improved from 1.70562 to 1

In [28]:
# set up a random seed for starting
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]

print("INPUT SEED:")
print("\"", ''.join([int_to_char[val] for val in pattern]), "\"")
print()
# generate characters from the generated output of LSTM
for i in range(2000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x/float(n_vocabs)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1: len(pattern)]
print("\nTHE END.")

INPUT SEED:
" i know i've got to try  
and make it wit "

h the world  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the soad  
the sound of the